In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-20 22:42:19,041 utils 400 [INFO]    [logger_func] start 


In [4]:
#========================================================================
# threshold scoring
def get_part_of_score(df, pred_col, thres_col, threshold):
    
    y_pred  = df.loc[df[thres_col]>=threshold, pred_col].values
    y_train = df.loc[df[thres_col]>=threshold, target].values
    upper_score = np.sqrt(mean_squared_error(y_train, y_pred))
    
    y_pred  = df.loc[df[thres_col]<threshold, pred_col].values
    y_train = df.loc[df[thres_col]<threshold, target].values
    lower_score = np.sqrt(mean_squared_error(y_train, y_pred))
    
    return upper_score, lower_score
#========================================================================
    
#========================================================================
# Outlier Postprocessing
def get_submit(base, is_blend=False, min_thres=0):

    # Classifier Thresfold
    if not(is_blend):
        
        pred = 'prediction'
        col_list = []
        score_list = []
        #========================================================================
        # base pred
        base_cols = [col for col in base.columns if col.count('base_')]
        base[pred] = base[base_cols]
        base['base_pred'] = base[base_cols]
        
        upper_score, lower_score = get_part_of_score(df=base[~base[target].isnull()], pred_col=pred, thres_col='clf_pred', threshold=min_thres)
        print(f"Base Upper Score: {upper_score} | Base Lower Score: {lower_score}")
        col_list += ['base_upper', 'base_lower']
        score_list += [upper_score, lower_score]
        #========================================================================
        
        #========================================================================
        # Remove Out
        if len(rm_out_cols):
            rm_out_pred = 'rm_out_pred'
            base[rm_out_pred] = base[rm_out_cols[0]]
            if min_thres>0:
                base.loc[base['clf_pred']<min_thres, pred] = base.loc[base['clf_pred']<min_thres, rm_out_pred]
                base.loc[base['no_out_flg']==1, pred] = base.loc[base['no_out_flg']==1, rm_out_pred]
        #     base.loc[base[pred].isnull(), pred] = base.loc[base[pred].isnull(), clf_out_cols[0]]
#             base.loc[base[pred].isnull(), pred] = base.loc[base[pred].isnull(), 'base_pred']
        
            upper_score, lower_score = get_part_of_score(df=base[~base[target].isnull()], pred_col=rm_out_pred, thres_col='clf_pred', threshold=min_thres)
            print(f"Rm Out Upper Score: {upper_score} | Rm Out Lower Score: {lower_score}")
            col_list += ['rm_out_upper', 'rm_out_lower']
            score_list += [upper_score, lower_score]
        #========================================================================
        
    # Simple Blender 
    else:
        base[pred] = base[pred_cols].mean(axis=1)
        out_cols = []
        min_thres = 0
    
    train = base[~base[target].isnull()]
    y_pred = train[pred].values
    y_train = train[target].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    
    #=======================================================================
    # Out Score 
    train.reset_index(inplace=True)
    out_score = get_part_of_score(df=train, pred_col=pred, thres_col=target, threshold=-30)
    #========================================================================
    print(f'''
    #========================================================================
    # CV SCORE AVG: {score}
    # OUT SCORE: {out_score}
    #========================================================================''')
    col_list += ['cv_score', 'out_score']
    score_list += [score, out_score]
    result = pd.Series(index=col_list, data=score_list)
    
    return result

#========================================================================

In [22]:
# Data Load
is_clf_out = [True, False][1]
is_no_out_flg = [True, False][1]
is_rm_out = [True, False][0]
rm_out_cols = []
clf_out_cols = []
#========================================================================
# Base Model Load
base = utils.read_df_pkl('../input/base_no_out_clf.gz')[[key, target, 'clf_pred', 'no_out_flg']].set_index(key)
alijs_type = pd.read_csv('../input/card_ids_grouping.csv').set_index(key)
base['type'] = alijs_type['type']
ens_list = glob.glob('../ensemble/lgb_ensemble/*.gz')
# ens_list = ['../model/LB3665_set/0213_152_stack_ridge_set-all_lgb27_NN0_other0_OUT29.7052_CV3.616445337563796_LB.gz']
# ens_list = ['../model/LB3664_set/0214_074_stack_ridge_set-all_lgb20_NN0_other0_OUT29.7295_CV3.618401419254512_LB.gz']
# lb3664_list = ['../ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz']
ens_list = glob.glob('../stack/0220_222_stack_ridge_set-all_valid-term-seed328_lgb41_NN0_ridge0_ext0_rmf0_OUT29.6294_CV3.611435147657334_LB.gz')
for path in ens_list:
    try:
        cv = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
    except AttributeError:
        print(path)
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
    
#     try:
#         blend = utils.read_df_pkl(path).set_index(key)['pred_mean']
#     except KeyError:
#         blend = utils.read_df_pkl(path).set_index(key)['prediction']
    blend = utils.read_df_pkl(path)[[key, 'prediction']].set_index(key)
    base[f'base_{cv}'] = blend
    
base_cols = [col for col in base.columns if col.count('base_')]
if len(base_cols)>1:
#     blend = utils.read_df_pkl(lb3664_list[0])[[key, 'prediction']].set_index(key)
#     base[f'lb3664'] = blend
    base['base_pred'] = base[base_cols].mean(axis=1)
#     base['base_pred'] = base['base_pred']*0.2 + base['lb3664']*0.8
    base.drop(base_cols, axis=1, inplace=True)
    base_cols = [col for col in base.columns if col.count('base_')]
#========================================================================

#========================================================================
# Clf Out Model
if is_clf_out:
    out_list = glob.glob('../clf_min_thres_ensemble/*.gz')
    out_list = ['../stack/0215_215_clf_out_lgb_out_part-clf_out_valid-ods_foldseed328_ESET0_row199950_lr0.01_235feats_3seed_70leaves_colsample0.325582_iter1222_OUT0_CV7-9325325611862105_LB.gz']
    for path in out_list:
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        base[f"clf_out_{cv}"] = blend
    clf_out_cols = [col for col in base.columns if col.count('clf_out_2')]
#========================================================================

#========================================================================
# No Out Flg Model
if is_no_out_flg:
    no_out_flg_list = glob.glob('../no_out_flg_ensemble/*.gz')
    no_out_flg_list = ['../stack/']

    for path in no_out_flg_list:
        cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        base[f"no_out_flg_{cv}"] = blend
    no_out_cols = [col for col in base.columns if col.count('no_out_flg_1')]
#========================================================================

#========================================================================
# No Out Model
if is_rm_out:
    rm_out_list = ['../stack/0220_224_stack_ridge_set-rm_out_valid-term-seed328_lgb5_NN0_ridge0_ext0_rmf0_OUT0_CV1.5459512546222733_LB.gz']
#     rm_out_list = ['../stack/dir_level2/0215_083_stack_ridge_set-rm_out_lgb20_NN0_other0_OUT0_CV1.5464357357745486_LB.gz']
    rm_out_list += ['../stack/dir_level2/0215_230_stack_ridge_set-rm_out_lgb20_NN0_ridge1_ext1_rmf1_OUT0_CV8.408986180641215_LB.gz']
#     rm_out_list = ['../ensemble/rm_outlier_ensemble/0215_222_no_out_lgb_out_part-no_out_valid-term_foldseed328_ESET0_row199710_lr0.01_235feats_3seed_70leaves_colsample0.325582_iter1944_OUT0_CV8-41402353053297_LB.gz']
#     rm_out_list = glob.glob('../ensemble/rm_outlier_ensemble/*.gz')

    for path in rm_out_list:
        if path.count('NN') and path.count('all'):continue
        try:
            cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        except AttributeError:
            print(path)
            sys.exit()
        try:
            blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
        if path.count('lgb'):
            col = f"lgb_rm_out_{cv}"
        elif path.count('NN'):
            col = f"nn_rm_out_{cv}"
        base[col] = blend
#========================================================================


#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf = utils.read_pkl_gzip('../stack/0215_204_stack_lgr_set-all_lgb20_NN0_ridge1_ext1_rmf1_OUT0_CV0.9541067640604712_LB.gz')[[key, 'prediction']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf['clf_pred'] =  clf['pred_mean']
# clf['clf_pred'] =  clf['prediction']
#========================================================================

#========================================================================
# Indexをそろえる
base['clf_pred'] = clf['clf_pred']
pred_cols = [col for col in base.columns if col.count('-') or col.count('base_pred')]
ignore_list += [col for col in base.columns if col.count('hist') or col.count('new_') or col.count('month')]
pred_cols = sorted(pred_cols)
base = base[[target, 'clf_pred', 'no_out_flg'] + pred_cols]

base_col_list = [col for col in base.columns if col.count('base_3-6')]
rm1_col_list = [col for col in base.columns if col.count('rm_out_1')]
rm8_col_list = [col for col in base.columns if col.count('rm_out_8')]
pred_col_list = base_col_list + rm1_col_list + rm8_col_list
if len(rm1_col_list):
    base.loc[base[rm1_col_list[0]].isnull(), rm1_col_list[0]] = base.loc[base[rm1_col_list[0]].isnull(), rm8_col_list[0]]
    base.drop(rm8_col_list[0], axis=1, inplace=True)
rm_out_cols = [col for col in base.columns if col.count('rm_out')]

base.head()
#========================================================================

100%|██████████| 1/1 [00:00<00:00,  9.36it/s]


,target,clf_pred,no_out_flg,base_3-611435147657334,lgb_rm_out_1-5459512546222733_LB
card_id,,,,,
C_ID_92a2005557,-0.820283,0.000444,1.0,-0.293175,-0.345815
C_ID_3d0044924f,0.392913,0.007831,0.0,-0.679402,0.288418
C_ID_d639edf6cd,0.688056,0.004074,0.0,0.616642,0.843978
C_ID_186d6a6901,0.142495,0.000797,0.0,0.200233,0.202080
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0,-0.106286,-0.253839


In [24]:
pred = 'prediction'
min_thres = 0.01
# min_thres = 0.019
result_list = []
clf_out_cols = []
df_score = get_submit(base=base, min_thres=min_thres)
# df_score.name = i
result_list.append(df_score)
df_score = pd.concat(result_list, axis=1)
display(df_score)

cv_score = df_score.loc['cv_score'].values[0]
out_score = df_score.loc['out_score'].values[0]

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend_base{len(base_cols)}_clf_out{len(clf_out_cols)}_rm_out{len(rm_out_cols)}_no_out_blend0_minthres{min_thres}_OUT{str(out_score[0])[:6]}-{str(out_score[1])[:6]}_CV{str(cv_score)[:7]}_LB')
display(submit.head())

Base Upper Score: 7.3636612195773665 | Base Lower Score: 2.1733631791436387
Rm Out Upper Score: 7.905665537655115 | Rm Out Lower Score: 2.161670361474217

    #========================================================================
    # CV SCORE AVG: 3.605588677952002
    # OUT SCORE: (1.8505941271670354, 29.656154392731857)
    #========================================================================


,0
base_upper,7.36366
base_lower,2.17336
rm_out_upper,7.90567
rm_out_lower,2.16167
cv_score,3.60559
out_score,"(1.8505941271670354, 29.656154392731857)"


,target
card_id,
C_ID_0ab67a22ab,-1.382863
C_ID_130fd0cbdd,-0.164823
C_ID_b709037bc5,-1.661338
C_ID_d27d835a9f,-0.086049
C_ID_2b5e3df5c2,-1.405824


In [31]:
pred_col_list = [target, 'clf_pred', 'no_out_flg', 'base_pred', 'rm_out_pred', 'prediction']
base = base[pred_col_list].rename(columns={'base_pred':'With_Outlier_41LGB_corr~0.985_Stack_OUT29.6294_CV3.611435', 'rm_out_pred':'Without_Outlier_5LGB_corr~0.985_Stack_CV1.54595', 'prediction':'LB3.662_OUT29.656_CV3.60558_WituoutOut1.85059'})
# utils.to_pkl_gzip(obj=base, path='../output/0220_234_elo_OOF_LB3662_and_2Stack_OOF')
base

,target,clf_pred,no_out_flg,With_Outlier_41LGB_corr~0.985_Stack_OUT29.6294_CV3.611435,Without_Outlier_5LGB_corr~0.985_Stack_CV1.54595,LB3.662_OUT29.656_CV3.60558_WituoutOut1.85059
card_id,,,,,,
C_ID_92a2005557,-0.820283,0.000444,1.0,-0.293175,-0.345815,-0.345815
C_ID_3d0044924f,0.392913,0.007831,0.0,-0.679402,0.288418,0.288418
C_ID_d639edf6cd,0.688056,0.004074,0.0,0.616642,0.843978,0.843978
C_ID_186d6a6901,0.142495,0.000797,0.0,0.200233,0.202080,0.202080
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0,-0.106286,-0.253839,-0.253839
C_ID_0894217f2f,0.871585,0.043221,0.0,-1.548723,-0.648290,-1.548723
C_ID_7e63323c00,0.230129,0.000891,0.0,-0.213595,-0.222721,-0.222721
C_ID_dfa21fc124,2.135850,0.001918,1.0,0.320844,0.031533,0.031533
C_ID_fe0fdac8ea,-0.065406,0.007651,0.0,0.311255,0.425661,0.425661


In [103]:
utils.to_pkl_gzip(obj=base.reset_index()[[key, target, 'prediction']], path='../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664')

In [ ]:
# #========================================================================
# # Classifier Pred Layer Scoring
pd.set_option('max_rows', 300)
base_col_list = [col for col in base.columns if col.count('base_3-6')]
rm_col_list = [col for col in base.columns if col.count('rm_out_')]
pred_col_list = base_col_list + rm_col_list

df = base[~base[target].isnull()]
df['clf_layer'] = df['clf_pred'].map(lambda x: np.round(x, 3))
layer_list = sorted(list(df['clf_layer'].value_counts().index))
df_list = []

for pred_col in pred_col_list:
    part_score_list = []
    df.loc[df['no_out_flg']==1, pred_col] = df.loc[df['no_out_flg']==1, rm1_col_list[0]]
    for layer in layer_list:
        tmp = df[df['clf_layer']==layer]
        y_pred = tmp[pred_col]
        y_train = tmp[target]
        try:
            part_score = np.sqrt(mean_squared_error(y_train, y_pred))
            part_score_list.append(part_score)
        except ValueError:
            part_score_list.append(0)
            continue
        
    tmp_score = pd.Series(data=part_score_list, index=layer_list, name=pred_col)
    df_list.append(tmp_score)
    
df_layer_score = pd.concat(df_list + [df['clf_layer'].value_counts().to_frame()], axis=1)
df_layer_score['diff_score'] = df_layer_score[pred_col_list[0]] - df_layer_score[pred_col_list[1]]
display(df_layer_score)
#========================================================================

In [405]:
#========================================================================
# Classifier Blend
from sklearn.metrics import roc_auc_score
train = base[~base[target].isnull()]
test = base[base[target].isnull()]
y_train = train[target].map(lambda x: 1 if x<-30 else 0).values
base_cols = [col for col in train.columns if col.count('base_')]
y_pred = (train[base_cols[6]]*0.5 + train[base_cols[8:10]].mean(axis=1)*0.4 + train[base_cols[10]]*0.1).values
roc_auc_score(y_train, y_pred)
#========================================================================

#========================================================================
# Manual Pred Threshold
clf_stack = utils.read_pkl_gzip('../stack/0215_204_stack_lgr_set-all_lgb20_NN0_ridge1_ext1_rmf1_OUT0_CV0.9541067640604712_LB.gz')[[key, 'prediction']].set_index(key)
tmp = base.copy()
tmp[target] = tmp[target].map(lambda x: 1 if x<-30 else 0)
tmp['clf_stack_pred'] = clf_stack['prediction']
tmp['clf_pred'] = tmp['base_0-9099420278047783_235features']
tmp.sort_values(by='clf_stack_pred', inplace=True, ascending=False)
# tmp.sort_values(by='clf_pred', inplace=True, ascending=False)
clf_stack_top100_id = list(tmp.iloc[:100, 0].index)
#========================================================================

SystemExit: 